In [23]:
import csv
import datetime as dt
import os
import os.path
import re
import math

import pandas as pd
import numpy as np
from django.core.management import call_command
from django.core.management.base import BaseCommand

from core.models import *

In [2]:
dir = "../working_files/csv_data/"

In [3]:
filename = os.path.join(dir, "cfs_2014_inmain.csv")
df = pd.read_csv(filename, encoding='ISO-8859-1', dtype={"streetno": "object"})

In [4]:
df.head()

,inci_id,calltime,calldow,case_id,callsource,primeunit,firstdisp,streetno,streetonly,street,...,secs2tr,secsar2tr,lastclr,secs2lc,secsar2lc,secstr2lc,timeclose,reptaken,closecode,closecomm
0,2014000002,1/1/14 0:00:22,4,NaN,PHONE,BK2,BK2,301,S ELM ST,301 S ELM ST,...,0,0,1/1/14 0:04:20,238,0,0,1/1/14 0:04:22,,10,NaN
1,2014000003,1/1/14 0:00:40,4,14000001,SELF,B200,B200,1610,GUESS RD,1610 GUESS RD,...,0,0,1/1/14 0:15:57,918,917,0,1/1/14 0:15:59,B200,1,NaN
2,2014000004,1/1/14 0:01:09,4,NaN,E911,B100,B100,3118,WEDGEDALE DR,3118 WEDGEDALE DR,...,0,0,1/1/14 0:29:20,1692,0,0,1/1/14 0:29:22,,10,NaN
3,2014000005,1/1/14 0:01:48,4,NaN,E911,B300,B300,1011,MORELAND AVE,1011 MORELAND AVE,...,0,0,1/1/14 0:04:07,139,0,0,1/1/14 0:04:08,,10,NaN
4,2014000006,1/1/14 0:01:54,4,NaN,E911,B300,B300,1299,CORNELL ST/W LAKEWOOD AVE,CORNELL ST/W LAKEWOOD AVE,...,0,0,1/1/14 0:04:54,180,0,0,1/1/14 0:04:55,,10,NaN


In [9]:
df.district.values

array(['D5  ', 'D2  ', 'D1  ', ..., 'D3  ', 'D3  ', 'D1  '], dtype=object)

In [ ]:
def isnan(x):
    return type(x) == float and math.isnan(x)

In [35]:
cities = [city.strip() for city in pd.unique(df.citydesc.values) if city and not isnan(city) and city.strip() != '']
sectors = [sector.strip() for sector in pd.unique(df.ra.values) if sector and not isnan(sector) and sector.strip() != '']

In [37]:
def unique_clean_values(column):
    return [x.strip() 
            for x in pd.unique(column.values)
            if x and not isnan(x) and x.strip()]

In [38]:
unique_clean_values(df.citydesc)

['DURHAM',
 'CHAPEL HILL',
 'CARY',
 'PERS',
 'ORAN',
 'MORRISVILLE',
 'RALEIGH',
 'BAHAMA',
 'RESEARCH TRIANGLE PA',
 'CARBORO',
 'CREEDMOOR',
 'WAKE',
 'STEM',
 'ROUGEMONT',
 'APEX',
 'CHATHAM COUNTY']

In [39]:
unique_clean_values(df.ra)

['NTH',
 'STH',
 'DSO',
 'WAKE',
 'PERS',
 'ORAN',
 'CH',
 'RAL',
 'CARR',
 'CARY',
 'GRAN',
 'MORR',
 'CHAT']

In [48]:
units = list(df.primeunit.values) + list(df.firstdisp.values) + list(df.reptaken.values)

In [49]:
xs = {unit.strip() for unit in units if unit and not isnan(unit) and unit.strip()}

In [50]:
len(xs)

790

In [52]:
df.iloc[0:10]

,inci_id,calltime,calldow,case_id,callsource,primeunit,firstdisp,streetno,streetonly,street,...,secs2tr,secsar2tr,lastclr,secs2lc,secsar2lc,secstr2lc,timeclose,reptaken,closecode,closecomm
0,2014000002,1/1/14 0:00:22,4,NaN,PHONE,BK2,BK2,301,S ELM ST,301 S ELM ST,...,0,0,1/1/14 0:04:20,238,0,0,1/1/14 0:04:22,,10,NaN
1,2014000003,1/1/14 0:00:40,4,14000001,SELF,B200,B200,1610,GUESS RD,1610 GUESS RD,...,0,0,1/1/14 0:15:57,918,917,0,1/1/14 0:15:59,B200,1,NaN
2,2014000004,1/1/14 0:01:09,4,NaN,E911,B100,B100,3118,WEDGEDALE DR,3118 WEDGEDALE DR,...,0,0,1/1/14 0:29:20,1692,0,0,1/1/14 0:29:22,,10,NaN
3,2014000005,1/1/14 0:01:48,4,NaN,E911,B300,B300,1011,MORELAND AVE,1011 MORELAND AVE,...,0,0,1/1/14 0:04:07,139,0,0,1/1/14 0:04:08,,10,NaN
4,2014000006,1/1/14 0:01:54,4,NaN,E911,B300,B300,1299,CORNELL ST/W LAKEWOOD AVE,CORNELL ST/W LAKEWOOD AVE,...,0,0,1/1/14 0:04:54,180,0,0,1/1/14 0:04:55,,10,NaN
5,2014000007,1/1/14 0:02:20,4,NaN,PHONE,B100,B100,2758,MANSFIELD AVE,2758 MANSFIELD AVE,...,0,0,1/1/14 0:29:27,1628,0,0,1/1/14 0:29:31,,10,NaN
6,2014000008,1/1/14 0:02:39,4,NaN,ALARM,B314,B314,68,BEVERLY DR,68 BEVERLY DR,...,0,0,1/1/14 0:17:00,862,126,0,1/1/14 0:17:08,,16,NaN
7,2014000011,1/1/14 0:03:45,4,NaN,E911,B212,B212,1200,CLARENDON ST,1200-BLK CLARENDON ST,...,0,0,1/1/14 0:26:15,1351,209,0,1/1/14 0:26:26,,16,NaN
8,2014000012,1/1/14 0:04:19,4,,W911,B310,B322,3311,SHANNON RD,3311 SHANNON RD,...,0,0,1/1/14 0:21:18,1018,741,0,1/1/14 0:21:19,,10,
9,2014000013,1/1/14 0:04:21,4,NaN,W911,B400,B400,620,RESERVOIR ST,620 RESERVOIR ST,...,0,0,1/1/14 0:07:10,170,0,0,1/1/14 0:07:12,,10,NaN


In [53]:
len(df)

373460

In [55]:
[c for c in df.iloc[0:10]]

['inci_id',
 'calltime',
 'calldow',
 'case_id',
 'callsource',
 'primeunit',
 'firstdisp',
 'streetno',
 'streetonly',
 'street',
 'citydesc',
 'zip',
 'crossroad1',
 'crossroad2',
 'geox',
 'geoy',
 'service',
 'agency',
 'statbeat',
 'district',
 'ra',
 'business',
 'naturecode',
 'nature',
 'priority',
 'rptonly',
 'cancelled',
 'notes',
 'timeroute',
 'secs2rt',
 'timefini',
 'secs2fn',
 'firstdtm',
 'secs2di',
 'secsrt2dsp',
 'secsfi2dsp',
 'firstenr',
 'secs2en',
 'secsdi2en',
 'firstarrv',
 'secs2ar',
 'secsdi2ar',
 'firsttran',
 'secs2tr',
 'secsar2tr',
 'lastclr',
 'secs2lc',
 'secsar2lc',
 'secstr2lc',
 'timeclose',
 'reptaken',
 'closecode',
 'closecomm']

In [70]:
batch = df.iloc[0:10]
[c for idx, c in batch.iterrows()]

[inci_id                                              2014000002
 calltime                                         1/1/14 0:00:22
 calldow                                                       4
 case_id                                                     NaN
 callsource                                                PHONE
 primeunit                                                BK2   
 firstdisp                                            BK2       
 streetno                                                    301
 streetonly                                             S ELM ST
 street                                             301 S ELM ST
 citydesc                                                 DURHAM
 zip                                                       27701
 crossroad1                                            WALKER ST
 crossroad2                                         HENDERSON ST
 geox                                                    2031759
 geoy                    

In [67]:
df.cancelled

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        1
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
373430    0
373431    1
373432    0
373433    0
373434    0
373435    0
373436    0
373437    1
373438    0
373439    0
373440    0
373441    0
373442    0
373443    0
373444    1
373445    0
373446    0
373447    1
373448    1
373449    0
373450    1
373451    0
373452    0
373453    0
373454    0
373455    1
373456    0
373457    0
373458    0
373459    0
Name: cancelled, dtype: float64

In [68]:
bool(0)

False

In [69]:
bool(1)

True

In [71]:
df.business

0                  EDGEMONT ELMS APT`S
1                                  NaN
2                                  NaN
3                                  NaN
4                                  NaN
5                                  NaN
6                                  NaN
7                                  NaN
8                                     
9                                  NaN
10                   OAK CREEK VILLAGE
11                                 NaN
12                                 NaN
13                                 NaN
14                 EDGEMONT ELMS APT`S
15                   CHAPEL TOWER APTS
16                                 NaN
17                                    
18        MCDOUGALD TERRACE - BLDG. 38
19                                 NaN
20                       COLONIAL APTS
21                                 NaN
22                                 NaN
23                                 NaN
24                                 NaN
25                       

In [72]:
def isnan(x):
    return type(x) == float and math.isnan(x)

In [73]:
isnan(df.iloc[1].business)

True

In [75]:
from itertools import chain

def flatmap(f, items):
    return chain.from_iterable(map(f, items))

In [83]:
def split_notes(notes):
    """
    Return a list of tuples.  Each tuple represents a single note and contains the corresponding call_id,
    the timestamp, the note-taker, and the text of the note.
    """
    tuples = []
    if notes is None:
        return []
    try:
        regex_split = re.findall(timestamp_expr, notes)
    except TypeError as ex:
        print(notes)
        raise ex
    for tup in regex_split:
        text = tup[0].split()
        text = text if text else None  # turn blanks into null
        try:
            timestamp = dt.datetime.strptime(tup[1], "%m/%d/%y %H:%M:%S")
        except ValueError:  # 4 digit year
            timestamp = dt.datetime.strptime(tup[1], "%m/%d/%Y %H:%M:%S")
        author = tup[2] if tup[2] else None
        tuples.append((text, timestamp, author))
    return tuples

In [85]:
list(flatmap(split_notes, df.notes.fillna('').values))[0:10]

[(['[EPD]',
   'Questions:',
   'It',
   'is',
   'not',
   'known',
   'who',
   'the',
   'suspect',
   'is.',
   'No',
   'one',
   'is',
   'reported',
   'to',
   'be',
   'in',
   'danger.',
   'No',
   'one',
   'needs',
   'medical',
   'attention.'],
  datetime.datetime(2014, 1, 1, 0, 1, 39),
  'ALLISONS'),
 (['[EPD]',
   'RESPONSE:',
   'PRIORITY',
   '3',
   'Questions:',
   'The',
   'caller',
   'is',
   'on',
   'scene.',
   'The',
   'shots',
   'were',
   'coming',
   'from:',
   'S',
   'Elm/Angier/E',
   'main',
   'The',
   'caller',
   'didn`t',
   'see',
   'or',
   'hear',
   'anything',
   'else.',
   'The',
   'caller',
   'is',
   'the',
   'victim.',
   'This',
   'incident',
   'is',
   'in',
   'progress.',
   'This',
   'is',
   'not',
   'a',
   'referral',
   'call.',
   'A',
   'gun',
   'is',
   'involved:',
   'Skipped',
   'by',
   'user',
   'The',
   'location',
   'of',
   'the',
   'weapons',
   'is',
   'not',
   'known.',
   'Shots',
   'have',


In [80]:
timestamp_expr = re.compile(
    "(.*?)\[(\d{2}/\d{2}/(?:\d{2}|\d{4}) \d{2}:\d{2}:\d{2}) (.*?)\]")

In [82]:
df.notes

0         [EPD] Questions: It is not known who the suspe...
1         272.09 ft from 1610 GUESS RD  [01/01/2014 00:0...
2         gave 10-14  [01/01/14 00:12:54 PAGEM]  NO FURT...
3         Linked Events 2014-000005(567) to 2014-000006(...
4         another caller advised same per charles ph #91...
5         gave 10-14  [01/01/14 00:13:03 PAGEM]  [EPD] Q...
6         (B314) will adv  [01/01/14 00:06:12 DILLARD]  ...
7         addlt caller 919-685-2883 advising 20 shots. n...
8         [FIRE] (M5) CANCEL PD  [01/01/14 00:10:09 CHAS...
9         [EPD] Questions: It is not known who the suspe...
10        add call stating they are still shooting  [01/...
11        [C115-TRANSPORT] {C115} 1 MALE 72  [01/01/14 0...
12        gave 10-14  [01/01/14 00:13:10 PAGEM]  NO COMP...
13        [LAW] (B100) scene is secure  [01/01/14 00:07:...
14        [EPD] Questions: The location of the suspect/p...
15        TELEPHONE COMPLAINANT*******  [01/01/14 00:10:...
16        NO COMP TO SEE  [01/01/14 00:1

In [89]:
def safe_strip(str_):
    if isnan(str_):
        return ''
    try:
        return str_.strip()
    except AttributeError:
        return str_
    
def isnan(x):
    return type(x) == float and math.isnan(x)    
    
def load_call_log():
    months = ("jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec")
    dfs = []
    for month in months:
        filename = os.path.join(dir, "cfs_{}2014_incilog.csv".format(month))
        if not os.path.isfile(filename):
            continue
        dfs.append(pd.read_csv(filename, encoding='ISO-8859-1'))
    df = pd.concat(dfs)

    string_cols = list(df.select_dtypes(include=['object']).columns)
    for col in string_cols:
        df[col] = df[col].apply(lambda x: safe_strip(x))
    return df

In [90]:
df = load_call_log()

In [91]:
df.head()

,incilogid,transtype,descript,timestamp,timeinsecs,inci_id,unitcode,radorev,unitperid,closecode
0,52983203,RPTO,Report Only,1/24/14 9:42:32,34952,2012372969,A221,R,758154,
1,52538477,D,Dispatched,1/4/14 20:02:02,72121,2013490611,E11,R,903734,
2,52538485,C,Cleared,1/4/14 20:02:06,72125,2013490611,E11,R,903734,1
3,52453784,C,Cleared,1/1/14 0:07:08,428,2013499964,B310,R,903821,1
4,52454517,X,Canceled,1/1/14 0:05:24,324,2013500009,B322,R,903825,


In [94]:
grouped = df[["transtype", "descript"]].groupby("transtype")

In [95]:
grouped.first()

,descript
transtype,
A,Arrived
A 2,
ACK,{M7} ACKNOWLEDGMENT OF IN
AMA,{M10} AGAINST MEDICAL ADV
AS,Assigned
AV,Available
As,Assigned
BORN,{E2} TIME OF BIRTH
C,Cleared


In [96]:
grouped.last()

,descript
transtype,
A,Arrived
A 2,
ACK,{M11} ACKNOWLEDGMENT OF I
AMA,{M3} AGAINST MEDICAL ADVI
AS,Assigned
AV,Available
As,Assigned
BORN,{M3} TIME OF BIRTH
C,Cleared


In [97]:
df['transtype'] = df['transtype'].map(lambda x: x.upper())

In [98]:
grouped = df[["transtype", "descript"]].groupby("transtype")
grouped.first()

,descript
transtype,
A,Arrived
A 2,
ACK,{M7} ACKNOWLEDGMENT OF IN
AMA,{M10} AGAINST MEDICAL ADV
AS,Assigned
AV,Available
BORN,{E2} TIME OF BIRTH
C,Cleared
CPR,{M12} CPR STARTED


In [107]:
for a, row in grouped.first().iterrows():
    print(a)
    print(row[0])

A
Arrived
A2

ACK
{M7} ACKNOWLEDGMENT OF IN
AMA
{M10} AGAINST MEDICAL ADV
AS
Assigned
AV
Available
BORN
{E2} TIME OF BIRTH
C
Cleared
CPR
{M12} CPR STARTED
CPRF
{E9} CPR STARTED
D
Dispatched
DXY
DispatchXY
E
En-Route
EMER
Emergency Alert
EXS
{M62} EXTRICATION STARTED
EXT
{E331} EXTRICATION TIME
EXTE
{LEB100} EXTRICATION TIME
FHR
Fire House Request
H
At Hospital
HIT
Potential Hit
L
Location Change
M
Misc. Radio
MILE
End Mileage
MOVE
Moved
PC
{M5} PATIENT CONTACT
PCF
{4SQ} PATIENT CONTACT FR
Q
At Quarters
R
Return to Qtr
RPTO
Report Only
S
Avail on Scene
STGE
{M62} STAGING  EMS UNIT
STGF
{E5} STAGING FIRE UNIT
T
Transport
UA
Un-Assigned
UC
EVENT UNDER CONTROL
UNDO
Arrived
X
Canceled


In [108]:
type(grouped.first())

pandas.core.frame.DataFrame

In [109]:
df.unitperid

0          758154
1          903734
2          903734
3          903821
4          903825
5          903825
6          903825
7          903825
8          903825
9          903825
10          49223
11         903803
12         903803
13         903827
14         903806
15         903806
16         903806
17         903806
18         903806
19         903806
20         903806
21         903804
22         903801
23         903823
24         903799
25         903799
26         903810
27         903819
28         903797
29         903789
           ...   
172632    1016757
172633    1016757
172634    1016765
172635    1016765
172636    1016765
172637    1016763
172638    1016735
172639    1016735
172640    1016741
172641    1016741
172642    1016731
172643    1016731
172644    1016741
172645    1016731
172646    1016735
172647    1011854
172648    1011854
172649    1011854
172650    1005761
172651    1005761
172652    1005761
172653    1016765
172654    1016765
172655    1016763
172656    